In [1]:
## this allows us to pull and push data from and to mongodb
import pymongo 
## allows us to use pandas
import pandas as pd 

In [2]:
 ## the mongo client is used to pass our credentials so that mongodb can verify that we have the right permissions 
from pymongo import MongoClient
client = MongoClient("mongodb+srv://<username>:<password>@sandbox.w7p98.mongodb.net/<database>?retryWrites=true&w=majority")


In [3]:
# connect to database
# i am using the sample_geospatial database that comes with mongodb
db = client["sample_geospatial"]
# connect to collection (a collection is the sql equivalent of a table)
col = db["shipwrecks"]
x = col.find_one()

In [4]:
## the output is in json
print(x)

{'_id': ObjectId('578f6fa2df35c7fbdbaed8c4'), 'recrd': '', 'vesslterms': '', 'feature_type': 'Wrecks - Visible', 'chart': 'US,U1,graph,DNC H1409860', 'latdec': 9.3547792, 'londec': -79.9081268, 'gp_quality': '', 'depth': '', 'sounding_type': '', 'history': '', 'quasou': '', 'watlev': 'always dry', 'coordinates': [-79.9081268, 9.3547792]}


In [5]:
# pass shipwrecks collection into pandas dataframe
df = pd.DataFrame(list(col.find()))

In [6]:
# take a sneak ppeak at the data
df.head()

,_id,recrd,vesslterms,feature_type,chart,latdec,londec,gp_quality,depth,sounding_type,history,quasou,watlev,coordinates
0,578f6fa2df35c7fbdbaed8c4,,,Wrecks - Visible,"US,U1,graph,DNC H1409860",9.354779,-79.908127,,,,,,always dry,"[-79.9081268, 9.3547792]"
1,578f6fa2df35c7fbdbaed8c5,,,Wrecks - Visible,"US,U1,graph,DNC H1409860",9.334030,-79.935722,,,,,,always dry,"[-79.9357223, 9.3340302]"
2,578f6fa2df35c7fbdbaed8c6,,,"Wrecks - Submerged, dangerous","US,U1,graph,DNC H1409860",9.356057,-79.907417,,,,,depth unknown,always under water/submerged,"[-79.9074173, 9.3560572]"
3,578f6fa2df35c7fbdbaed8c7,,,Wrecks - Visible,"US,U1,graph,DNC H1409860",9.339050,-79.913712,,,,,,always dry,"[-79.9137115, 9.3390503]"
4,578f6fa2df35c7fbdbaed8c8,,,"Wrecks - Submerged, dangerous","US,U1,graph,DNC H1409860",9.341881,-79.910385,,,,,depth unknown,always under water/submerged,"[-79.9103851, 9.3418808]"


In [7]:
## rows and columns of data frame
df.shape

(11095, 14)

In [8]:
## data tyoe pf each column
df.dtypes

_id               object
recrd             object
vesslterms        object
feature_type      object
chart             object
latdec           float64
londec           float64
gp_quality        object
depth             object
sounding_type     object
history           object
quasou            object
watlev            object
coordinates       object
dtype: object

In [9]:
## all unique values in the feature_type column
df.feature_type.unique()

array(['Wrecks - Visible', 'Wrecks - Submerged, dangerous',
       'Wrecks - Submerged, nondangerous', '',
       'distributed remains of wreck'], dtype=object)

In [11]:
## i want to see everything where the feature_type = Wrecks Visible
pipeline =[
    {
        "$match":
            {
                "feature_type":"Wrecks - Visible"
            }
    }
]

## aggregate results into one dataframe called df2
aggResult = col.aggregate(pipeline)
df2 = pd.DataFrame(list(aggResult))
df2.head()

,_id,recrd,vesslterms,feature_type,chart,latdec,londec,gp_quality,depth,sounding_type,history,quasou,watlev,coordinates
0,578f6fa2df35c7fbdbaed8c4,,,Wrecks - Visible,"US,U1,graph,DNC H1409860",9.354779,-79.908127,,,,,,always dry,"[-79.9081268, 9.3547792]"
1,578f6fa2df35c7fbdbaed8c5,,,Wrecks - Visible,"US,U1,graph,DNC H1409860",9.334030,-79.935722,,,,,,always dry,"[-79.9357223, 9.3340302]"
2,578f6fa2df35c7fbdbaed8c7,,,Wrecks - Visible,"US,U1,graph,DNC H1409860",9.339050,-79.913712,,,,,,always dry,"[-79.9137115, 9.3390503]"
3,578f6fa2df35c7fbdbaed8ca,,,Wrecks - Visible,"US,US,reprt,L-1083/14",9.364139,-79.940556,,,,,,always dry,"[-79.940556, 9.3641392]"
4,578f6fa2df35c7fbdbaed8cd,,,Wrecks - Visible,"US,US,reprt,L-1-2015",9.360139,-79.908139,,,,,,always dry,"[-79.9081389, 9.3601389]"


In [19]:
## use folium (leaflet.js)
import folium


world_map = folium.Map(location = [35,100], zoom_start = 4)

for i in range(len(df2)):
   folium.Marker(location = [df2.latdec[i],df2.londec[i]], icon=folium.Icon(color='red',icon='fa-ship', prefix='fa')).add_to(world_map)
    
world_map.save('world.html')